# Preference Alignment with Direct Preference Optimization (DPO)

This notebook will guide you through the process of fine-tuning a language model using Direct Preference Optimization (DPO). We will use the SmolLM2-135M-Instruct model which has already been through a SFT training, so it it compatible with DPO. You can also use the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
     <h2 style='margin: 0;color:blue'>Exercise: Aligning SmolLM2 with DPOTrainer</h2>
     <p>Take a dataset from the Hugging Face hub and align a model on it. </p> 
     <p><b>Difficulty Levels</b></p>
     <p>🐢 Use the `trl-lib/ultrafeedback_binarized` dataset</p>
     <p>🐕 Try out the `argilla/ultrafeedback-binarized-preferences` dataset</p>
     <p>🦁 Select a dataset that relates to a real-world use case you’re interested in, or use the model you trained in 
        <a href="../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb">1_instruction_tuning</a></p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login

login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

## Import libraries


In [2]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig

## Format dataset

In [23]:
# Load dataset

# TODO: 🦁🐕 change the dataset to one of your choosing
dataset = load_dataset(path="argilla/ultrafeedback-binarized-preferences", split="train")

In [24]:
# TODO: 🐕 If your dataset is not represented as conversation lists, you can use the `process_dataset` function to convert it.
print(dataset)
def process_dataset(sample):
    sample["prompt"] = sample.pop("instruction")
    sample["chosen"] = sample.pop("chosen_response")
    sample["rejected"] = sample.pop("rejected_response")
    
    return sample

dataset = dataset.map(process_dataset)
print(dataset)

Dataset({
    features: ['source', 'instruction', 'chosen_response', 'rejected_response', 'chosen_avg_rating', 'rejected_avg_rating', 'chosen_model'],
    num_rows: 63619
})


Map:   0%|          | 0/63619 [00:00<?, ? examples/s]

Dataset({
    features: ['source', 'chosen_avg_rating', 'rejected_avg_rating', 'chosen_model', 'prompt', 'chosen', 'rejected'],
    num_rows: 63619
})


## Select the model

We will use the SmolLM2-135M-Instruct model which has already been through a SFT training, so it it compatible with DPO. You can also use the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).


<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; width:80%; color:black'>
     <p>🦁 change the model to the path or repo id of the model you trained in <a href="../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb">1_instruction_tuning</a></p>
</div>


In [17]:
# TODO: 🦁 change the model to the path or repo id of the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb)

model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.float32,
).to(device)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-DPO-argilla-ultrafeedback"
finetune_tags = ["smol-course", "module_2"]

In [18]:
# Let's test the base model before training
prompt = "Write a Python program that uses the sieve of Eratosthenes to find prime numbers up to 100"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=300)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
system
You are a helpful AI assistant named SmolLM, trained by Hugging Face
user
Write a Python program that uses the sieve of Eratosthenes to find prime numbers up to 100
assistant
Here's a Python program that uses the Sieve of Eratosthenes to find prime numbers up to 100:

```python
def sieve_of_eratosthenes(n):
    sieve = [True] * (n + 1)
    sieve[0:2] = [False, False]  # 0 and 1 are not prime numbers

    for current_prime in range(2, int(n**0.5) + 1):
        if sieve[current_prime]:
            for multiple in range(current_prime**2, n + 1, current_prime):
                sieve[multiple] = False

    return [num for num, is_prime in enumerate(sieve) if is_prime]

# Example usage:
n = 100
primes = sieve_of_eratosthenes(n)
print(primes)
```

This program works by creating a boolean array `sieve` where `sieve[i]` represents whether `i` is a prime number or not. It then iterates over the numbers from 2 to the square root of `n`, marking as composite (not prime) any

## Train model with DPO

In [19]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# Training arguments
training_args = DPOConfig(
    # Training batch size per GPU
    per_device_train_batch_size=2,
    # Number of updates steps to accumulate before performing a backward/update pass
    # Effective batch size = per_device_train_batch_size * gradient_accumulation_steps
    gradient_accumulation_steps=2,
    # Saves memory by not storing activations during forward pass
    # Instead recomputes them during backward pass
    gradient_checkpointing=True,
    # Base learning rate for training
    learning_rate=5e-5,
    # Learning rate schedule - 'cosine' gradually decreases LR following cosine curve
    lr_scheduler_type="cosine",
    # Total number of training steps
    max_steps=1000,
    # Disables model checkpointing during training
    save_strategy="no",
    # How often to log training metrics
    logging_steps=1,
    # Directory to save model outputs
    output_dir="smol_dpo_output_turtle",
    # Number of steps for learning rate warmup
    warmup_steps=100,
    # Use bfloat16 precision for faster training
    bf16=True,
    # Disable wandb/tensorboard logging
    report_to="none",
    # Keep all columns in dataset even if not used
    remove_unused_columns=False,
    # Enable MPS (Metal Performance Shaders) for Mac devices
    use_mps_device=device == "mps",
    # Model ID for HuggingFace Hub uploads
    hub_model_id=finetune_name,
    # DPO-specific temperature parameter that controls the strength of the preference model
    # Lower values (like 0.1) make the model more conservative in following preferences
    beta=0.1,
    # Maximum length of the input prompt in tokens
    max_prompt_length=1024,
    # Maximum combined length of prompt + response in tokens
    max_length=1536,
)

In [25]:
trainer = DPOTrainer(
    # The model to be trained
    model=model,
    # Training configuration from above
    args=training_args,
    # Dataset containing preferred/rejected response pairs
    train_dataset=dataset,
    # Tokenizer for processing inputs
    processing_class=tokenizer,
    # DPO-specific temperature parameter that controls the strength of the preference model
    # Lower values (like 0.1) make the model more conservative in following preferences
    # beta=0.1,
    # Maximum length of the input prompt in tokens
    # max_prompt_length=1024,
    # Maximum combined length of prompt + response in tokens
    # max_length=1536,
)

Extracting prompt from train dataset:   0%|          | 0/63619 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/63619 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/63619 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [26]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

# Save to the huggingface hub if login (HF_TOKEN is set)
if os.getenv("HF_TOKEN"):
    trainer.push_to_hub(tags=finetune_tags)

  0%|          | 0/1000 [00:00<?, ?it/s]

/home/user/HuaProjects/Tookies/smol-course/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 0.6931, 'grad_norm': 48.13129806518555, 'learning_rate': 5.000000000000001e-07, 'rewards/chosen': 0.0, 'rewards/rejected': 0.0, 'rewards/accuracies': 0.0, 'rewards/margins': 0.0, 'logps/chosen': -173.53970336914062, 'logps/rejected': -139.02413940429688, 'logits/chosen': 3.3423714637756348, 'logits/rejected': 7.199538707733154, 'epoch': 0.0}
{'loss': 0.6931, 'grad_norm': 78.19709777832031, 'learning_rate': 1.0000000000000002e-06, 'rewards/chosen': 0.0, 'rewards/rejected': 0.0, 'rewards/accuracies': 0.0, 'rewards/margins': 0.0, 'logps/chosen': -457.0621032714844, 'logps/rejected': -448.9066467285156, 'logits/chosen': 3.4771037101745605, 'logits/rejected': 4.961572647094727, 'epoch': 0.0}
{'loss': 0.6931, 'grad_norm': 70.66542053222656, 'learning_rate': 1.5e-06, 'rewards/chosen': 0.040195465087890625, 'rewards/rejected': 0.03716712072491646, 'rewards/accuracies': 0.5, 'rewards/margins': 0.003028344362974167, 'logps/chosen': -483.64508056640625, 'logps/rejected': -208.83499145507

In [27]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a Python program that uses the sieve of Eratosthenes to find prime numbers up to 100"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.
outputs = model.generate(**inputs, max_new_tokens=500)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


/home/user/HuaProjects/Tookies/smol-course/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


After training:
system
You are a helpful AI assistant named SmolLM, trained by Hugging Face
user
Write a Python program that uses the sieve of Eratosthenes to find prime numbers up to 100
assistant
Here's a Python program that uses the Sieve of Eratosthenes to find prime numbers up to 100:

```python
def sieve_of_eratosthenes(n):
    sieve = [True] * (n + 1)
    sieve[0:2] = [False, False]  # 0 and 1 are not prime numbers

    for current_prime in range(2, int(n**0.5)+1):
        if sieve[current_prime]:
            for multiple in range(current_prime**2, n + 1, current_prime):
                sieve[multiple] = False

    return [num for num, is_prime in enumerate(sieve) if is_prime]

# Example usage:
n = 100
primes = sieve_of_eratosthenes(n)
print(primes)
```

This program works by iterating from 2 to the square root of `n`, marking as composite (not prime) the multiples of each prime number. The `range` function is used to generate a range of numbers from 2 to `n`, and the `is_prime`

In [ ]:
from IPython.display import display, HTML
display(HTML("<h2>Kernel will shut down...</h2>"))

import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True) 

{'status': 'ok', 'restart': True}

: 

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `DPOTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.